In [58]:
# Cell 1: Setup and Parse Stock List
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
import importlib
import script
importlib.reload(script)
from script import parse_stocks_file, get_signal_description

ROOT = Path.cwd()
stocks_file = ROOT / 'stocks.txt'

# Parse stocks.txt to get individual tickers and baskets
individual_tickers, baskets = parse_stocks_file(stocks_file)

print(f"Individual tickers: {individual_tickers}")
print(f"\nBaskets: {baskets}")

Individual tickers: ['TSLA', 'NVDA', 'MSFT', 'META', 'PLTR', 'MSTR', 'ASML', 'AMD', 'AVGO', 'ALAB', 'MRVL', 'BTC-USD', 'SOL-USD']

Baskets: {'Main AI Basket': ['TSLA', 'NVDA', 'MSFT', 'META', 'PLTR', 'ASML'], 'Secondary AI Basket': ['AMD', 'AVGO', 'ALAB', 'MRVL']}


In [59]:
# Cell 2: Run Batch Analysis
from script import analyze_ticker, analyze_basket

def run_batch(individual_tickers, baskets, concurrency=6, daily_bars=60, weekly_bars=52):
    """Run analysis on all individual tickers and baskets"""
    results = []
    
    # Analyze individual tickers in parallel
    with ThreadPoolExecutor(max_workers=concurrency) as ex:
        futures = {ex.submit(analyze_ticker, t, daily_bars, weekly_bars): t for t in individual_tickers}
        for fut in as_completed(futures):
            results.append(fut.result())
    
    # Analyze baskets (market cap weighted aggregations)
    for basket_name, constituents in baskets.items():
        basket_result = analyze_basket(basket_name, constituents, daily_bars, weekly_bars)
        results.append(basket_result)
    
    return pd.DataFrame(results)

# Run the analysis
df = run_batch(individual_tickers, baskets)
df = df.round(2)

# Print Summary Stats
print(f"\nAnalysis complete: {len(df)} rows ({len(individual_tickers)} tickers + {len(baskets)} baskets)")
print("FULL HOLD + ADD tickers:", ', '.join(df[df['signal'] == 'FULL HOLD + ADD']['ticker']))

df


Analysis complete: 15 rows (13 tickers + 2 baskets)
FULL HOLD + ADD tickers: TSLA, PLTR, NVDA, ASML, MRVL, [Main AI Basket]


,ticker,signal,current_price,price_note,date,d20,d50,d100,d200,w10,...,weekly_val,s1,s2,s3,r1,r2,r3,notes,confluence,recommendation
0,TSLA,FULL HOLD + ADD,438.07,last close,2026-01-04,464.27,445.01,418.45,360.20,442.77,...,219.94,411.45,382.78,373.04,488.54,474.07,470.75,,BALANCED,Enter on Dip
1,META,HOLD MOST → REDUCE,650.41,last close,2026-01-04,658.59,651.82,698.45,672.61,640.47,...,586.28,580.78,578.18,546.88,795.06,789.62,758.54,,BALANCED,No Buy
2,PLTR,FULL HOLD + ADD,167.86,last close,2026-01-04,184.55,180.94,176.47,151.04,179.10,...,69.16,147.56,142.34,128.51,207.52,190.00,188.20,,WEAK,Skip – Poor Setup
3,NVDA,FULL HOLD + ADD,188.85,last close,2026-01-04,182.96,186.50,183.02,160.48,183.82,...,86.60,176.75,169.54,164.05,212.18,NaN,NaN,,EXTENDED,Wait for Support
4,MSTR,CASH,157.16,last close,2026-01-04,167.75,202.40,267.75,320.20,179.39,...,235.84,155.61,120.23,113.69,543.00,457.22,430.35,,BALANCED,No Buy
5,MSFT,HOLD MOST → REDUCE,472.94,last close,2026-01-04,483.17,495.21,503.04,476.73,485.44,...,410.33,464.89,407.71,404.37,553.50,552.69,530.04,,BALANCED,No Buy
6,ASML,FULL HOLD + ADD,1163.78,last close,2026-01-04,1083.21,1056.14,965.47,843.33,1063.95,...,574.25,988.40,946.11,933.75,NaN,NaN,NaN,,WEAK,Skip – Poor Setup
7,AMD,HOLD,223.47,last close,2026-01-04,214.41,227.63,203.23,163.23,219.71,...,129.85,194.28,161.81,153.34,267.08,227.30,NaN,,BALANCED,No Buy
8,ALAB,HOLD MOST → REDUCE,179.56,last close,2026-01-04,162.40,160.76,179.74,134.57,161.72,...,107.54,148.51,131.42,84.78,262.90,201.86,199.47,,WEAK,No Buy
9,AVGO,HOLD,347.62,last close,2026-01-04,360.65,361.22,344.91,288.92,359.56,...,220.24,328.43,323.43,280.85,413.82,385.74,372.88,,WEAK,No Buy


In [22]:
# Cell 3: Print Buy Summary
def print_buy_summary(df, cash_available=118305):
    print("\n### Larsson Portfolio Buy Summary – Next Trading Day\n")
    print("**Rule-Based Only** — Conservative phased entry rules locked in:")
    print("- No immediate starters in Bullish zones (wait for better prices).")
    print("- Primary adds only on real weakness to predefined green supports.\n")
    
    print(f"**Cash Available**: ~${cash_available:,.0f} (~59% dry powder)\n")
    
    # Load target allocations from targets.csv
    targets_file = ROOT / 'targets.csv'
    if targets_file.exists():
        targets_df = pd.read_csv(targets_file)
        target_dict = dict(zip(targets_df['ticker'], targets_df['target_pct']))
        value_dict = dict(zip(targets_df['ticker'], targets_df['target_value']))
    else:
        target_dict = {}
        value_dict = {}
        print("⚠️  targets.csv not found - using N/A for target percentages\n")
    
    # Use exact match instead of contains
    eligible = df[df['signal'] == "FULL HOLD + ADD"]
    if eligible.empty:
        print("No FULL HOLD + ADD names — no buys recommended.")
        return
    
    def get_primary_zone(row):
        """Hybrid conservative primary add zone: Lower Value Area + Key Long-Term SMAs"""
        val = row['daily_val']
        poc = row['daily_poc']
        d100 = row['d100']
        d200 = row['d200']
        
        # Handle NaN/missing
        if pd.isna(val) or pd.isna(poc):
            return f"Near Key SMAs (D100 ${int(d100)} / D200 ${int(d200)})"
        
        lower_va = f"Lower Value Area (${int(val)}–${int(poc)})"
        sma_part = f"or Key Long-Term SMA (D100 ${int(d100)} / D200 ${int(d200)})"
        return f"{lower_va} {sma_part}"
    
    print("| Ticker | Target % | Current Price (Close) | Confluence | Buy Recommendation | Primary Add (40–50% of target) | Primary Add Zone (Conservative) | Approx Shares at Zone |")
    print("|--------|----------|-----------------------|------------|------------------------------|--------------------------------|---------------------------------|-----------------------|")
    
    for _, row in eligible.iterrows():
        ticker = row['ticker']
        price = row['current_price']
        confluence = row['confluence']
        rec = row['recommendation']
        
        # Get target % and value from config file
        target_pct = target_dict.get(ticker, 'N/A')
        target_val = value_dict.get(ticker, 4001)
        
        if target_pct != 'N/A':
            target_pct_str = f"{target_pct}%"
        else:
            target_pct_str = 'N/A'
        
        # Calculate primary add amounts (40-50% of target)
        primary_low = target_val * 0.4
        primary_high = target_val * 0.5
        
        # Get primary zone
        zone = get_primary_zone(row)
        
        # Share estimate using approximate zone midpoint (or current price fallback)
        zone_mid_est = price * 0.9  # rough 10% dip estimate for conservatism
        shares_low = int(primary_low / zone_mid_est)
        shares_high = int(primary_high / zone_mid_est)
        shares_str = f"{shares_low}–{shares_high} shares"
        
        print(f"| **{ticker}** | {target_pct_str} | ${price:.2f} | **{confluence}** | {rec} | ~${primary_low:,.0f}–${primary_high:,.0f} | {zone} | {shares_str} |")
    
    print("\n**No Buy Action**")
    print("- All other names: Not FULL HOLD + ADD or confluence insufficient.\n")
    print("**Execution Plan**")
    print("- No limits to place / Wait for weakness to primary zones / etc.")

print_buy_summary(df)



### Larsson Portfolio Buy Summary – Next Trading Day

**Rule-Based Only** — Conservative phased entry rules locked in:
- No immediate starters in Bullish zones (wait for better prices).
- Primary adds only on real weakness to predefined green supports.

**Cash Available**: ~$118,305 (~59% dry powder)

| Ticker | Target % | Current Price (Close) | Confluence | Buy Recommendation | Primary Add (40–50% of target) | Primary Add Zone (Conservative) | Approx Shares at Zone |
|--------|----------|-----------------------|------------|------------------------------|--------------------------------|---------------------------------|-----------------------|
| **PLTR** | 5% | $167.86 | **WEAK** | Skip – Poor Setup | ~$4,000–$5,000 | Lower Value Area ($169–$181) or Key Long-Term SMA (D100 $176 / D200 $151) | 26–33 shares |
| **NVDA** | 18% | $188.85 | **EXTENDED** | Wait for Support | ~$14,402–$18,002 | Lower Value Area ($175–$182) or Key Long-Term SMA (D100 $183 / D200 $160) | 84–105 shares |
| *

In [62]:
# Cell 4: Export Buy Summary to PDF
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.enums import TA_CENTER
from datetime import datetime, timedelta

def export_buy_summary_to_pdf(df, starting_cash=118305):
    """Generate a styled PDF report of the buy summary using reportlab"""
    
    # Load target allocations
    targets_file = ROOT / 'targets.csv'
    if targets_file.exists():
        targets_df = pd.read_csv(targets_file)
        target_dict = dict(zip(targets_df['ticker'], targets_df['target_pct']))
        value_dict = dict(zip(targets_df['ticker'], targets_df['target_value']))
    else:
        target_dict = {}
        value_dict = {}
    
    # Load current holdings
    holdings_file = ROOT / 'holdings.csv'
    holdings_dict = {}
    if holdings_file.exists():
        holdings_df = pd.read_csv(holdings_file)
        for _, row in holdings_df.iterrows():
            holdings_dict[row['ticker']] = {
                'shares': row['shares'],
                'avg_cost': row['avg_cost']
            }
    
    # Holdings are tracked for allocation %, but cash available is separate
    # (holdings already paid for separately)
    cash_available = starting_cash
    
    # Get eligible tickers (exclude baskets)
    eligible = df[(df['signal'] == "FULL HOLD + ADD") & (~df['ticker'].str.startswith('['))]
    
    # Calculate next trading day (skip weekends)
    today = datetime.now()
    next_trading_day = today + timedelta(days=1)
    # Skip Saturday (5) and Sunday (6)
    while next_trading_day.weekday() >= 5:
        next_trading_day += timedelta(days=1)
    
    # Generate PDF filename - save directly to Downloads (no timestamp, will overwrite)
    # Generate PDF filename with timestamp - save directly to Downloads
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    pdf_filename = f"buy_summary_{timestamp}.pdf"
    pdf_path = Path.home() / 'Downloads' / pdf_filename
    
    # Create PDF
    doc = SimpleDocTemplate(str(pdf_path), pagesize=letter,
                           rightMargin=30, leftMargin=30,
                           topMargin=30, bottomMargin=18)
    
    # Container for the 'Flowable' objects
    elements = []
    
    # Define styles
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=18,
        textColor=colors.HexColor('#2c3e50'),
        spaceAfter=20,
        alignment=TA_CENTER
    )
    
    # Add title with dates
    title_text = f"Larsson Portfolio Buy Summary – For {next_trading_day.strftime('%A, %B %d, %Y')}<br/><font size=12>(Data as of {today.strftime('%B %d')} Close)</font>"
    elements.append(Paragraph(title_text, title_style))
    elements.append(Spacer(1, 0.2*inch))
    
    # Calculate total portfolio value
    total_holdings_value = 0
    for ticker, holding in holdings_dict.items():
        if holding['shares'] > 0:
            current_price = df[df['ticker'] == ticker]['current_price'].values
            if len(current_price) > 0:
                total_holdings_value += holding['shares'] * current_price[0]
    
    total_portfolio_value = total_holdings_value + cash_available
    cash_pct = (cash_available / total_portfolio_value * 100) if total_portfolio_value > 0 else 0
    
    # Add summary info
    summary_text = f"""
    <b>Rule-Based Only</b> — Conservative phased entry rules:<br/>
    <br/>
    • <b>EXTENDED</b> Stocks: Wait for pullback to support (D100/D200)<br/>
    • <b>BALANCED</b> Stocks: Enter on dip to lower Value Area or key SMAs<br/>
    • <b>WEAK</b> Stocks: Skip until technical setup improves<br/>
    <br/>
    <b>Total Portfolio Value:</b> ${total_portfolio_value:,.0f}<br/>
    <b>Current Holdings:</b> ${total_holdings_value:,.0f}<br/>
    <b>Cash Available:</b> ${cash_available:,.0f} (~{cash_pct:.0f}% dry powder)
    """
    elements.append(Paragraph(summary_text, styles['Normal']))
    elements.append(Spacer(1, 0.3*inch))
    
    # Add Portfolio Health Check section (baskets as macro indicators)
    basket_rows = df[df['ticker'].str.startswith('[')]
    if not basket_rows.empty:
        basket_text = "<b>Portfolio Health Check (Macro View)</b><br/>"
        for _, basket_row in basket_rows.iterrows():
            basket_name = basket_row['ticker'].strip('[]')
            signal = basket_row.get('signal', 'UNKNOWN')
            
            # Get constituents from baskets dict
            constituents = baskets.get(basket_name, [])
            constituents_str = ', '.join(constituents) if constituents else 'N/A'
            
            # Get signal description
            description = get_signal_description(signal)
            
            basket_text += f"• <b>{basket_name}</b> ({constituents_str}): <b>{signal}</b><br/>"
            basket_text += f"<font size=9><i>{description}</i></font><br/><br/>"
        
        elements.append(Paragraph(basket_text, styles['Normal']))
        elements.append(Spacer(1, 0.3*inch))
    
    # Prepare table data with holdings awareness
    table_data = [['Ticker', 'Target %', 'Current %', 'Price', 'Confluence', 'Recommendation', 'Next Add', 'Zone', 'Shares']]
    if eligible.empty:
        # Add a "No Buy" row if no eligible tickers
        table_data.append(['No Eligible', 'N/A', 'N/A', 'N/A', 'N/A', 'No FULL HOLD + ADD signals', 'N/A', 'N/A', 'N/A'])
    else:
        for _, row in eligible.iterrows():
            ticker = row['ticker']
            price = row['current_price']
            confluence = row['confluence']
            rec = row['recommendation']
            
            # Get target data - calculate based on total portfolio value
            target_pct = target_dict.get(ticker, 'N/A')
            if target_pct != 'N/A':
                target_val = (target_pct / 100) * total_portfolio_value
            else:
                target_val = 4001
            target_pct_str = f"{target_pct}%" if target_pct != 'N/A' else 'N/A'
            
            # Get current holding
            holding = holdings_dict.get(ticker, {'shares': 0, 'avg_cost': 0})
            current_value = holding['shares'] * price
            current_pct = (current_value / total_portfolio_value) * 100 if total_portfolio_value > 0 else 0
            current_pct_str = f"{current_pct:.0f}%" if current_pct > 0 else "0%"
            
            # Calculate remaining gap to target
            remaining_gap = target_val - current_value
            
            # Calculate primary add amounts (40-50% of remaining gap, not total target)
            if remaining_gap > 0:
                primary_low = remaining_gap * 0.4
                primary_high = remaining_gap * 0.5
            else:
                # Already at or above target
                primary_low = 0
                primary_high = 0
            
            # Shorten recommendation if too long (updated for new confluence/recommendation labels)
            rec_short = rec.replace("Wait for Support", "Wait Support").replace("Enter on Dip", "Enter on Dip").replace("Skip – Poor Setup", "Skip")
            
            # For "Skip" recommendations, don't show entry zones or share counts
            if rec_short == "Skip":
                primary_add_str = "N/A"
                zone = "Poor Setup"
                shares_str = "N/A"
            else:
                # Get primary zone based on confluence
                val = row['daily_val']
                poc = row['daily_poc']
                d100 = row['d100']
                d200 = row['d200']
                
                # Different zones based on confluence state
                if confluence == "BALANCED":
                    # For balanced, use basket signal to determine zone preference
                    # Find which basket(s) contain this ticker
                    ticker_baskets = [name for name, constituents in baskets.items() if ticker in constituents]
                    
                    # Check basket strength
                    strong_basket = False
                    if ticker_baskets:
                        for basket_name in ticker_baskets:
                            basket_signal = df[df['ticker'] == f'[{basket_name}]']['signal'].values
                            if len(basket_signal) > 0 and basket_signal[0] == "FULL HOLD + ADD":
                                strong_basket = True
                                break
                    
                    # For balanced with strong basket, D100 is acceptable
                    # For balanced with weak/no basket, prefer Lower VA for more safety
                    if strong_basket:
                        zone = f">> D100 ${int(d100)}"
                        zone_mid_est = d100
                    else:
                        if not pd.isna(val) and val < price:
                            zone = f">> Lower VA ${int(val)}\nor D100 ${int(d100)}"
                            zone_mid_est = val  # Target lower VA
                        else:
                            zone = f"D100 ${int(d100)}"
                            zone_mid_est = d100
                elif confluence == "EXTENDED":
                    # For extended, ALWAYS prefer D200 - extended means technically stretched
                    # Basket strength = conviction to wait patiently, not justification to chase
                    zone = f">> D200 ${int(d200)}\nor D100 ${int(d100)}"
                    zone_mid_est = d200  # Always target deeper pullback for extended stocks
                else:
                    # Fallback for other states
                    if pd.isna(val) or pd.isna(poc):
                        zone = f"D100 ${int(d100)}"
                        zone_mid_est = d100
                    else:
                        zone = f"${int(val)}-${int(poc)}"
                        zone_mid_est = (val + poc) / 2
                
                # Share estimate
                shares_low = int(primary_low / zone_mid_est)
                shares_high = int(primary_high / zone_mid_est)
                shares_str = f"{shares_low}-{shares_high}"
                primary_add_str = f"${primary_low:,.0f}-\n${primary_high:,.0f}"
            
            table_data.append([
                ticker,
                target_pct_str,
                current_pct_str,
                f"${price:.2f}",
                confluence,
                rec_short,
                primary_add_str,
                zone,
                shares_str
            ])
    
    # Create table with updated column widths for new Current % column
    table = Table(table_data, colWidths=[0.9*inch, 0.65*inch, 0.65*inch, 0.55*inch, 0.8*inch, 1.3*inch, 0.75*inch, 1.0*inch, 0.5*inch])
    
    # Apply table style with confluence color coding
    style_commands = [
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#3498db')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 7),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ]
    
    # Add confluence color coding for each row
    if not eligible.empty:
        for i, (_, row) in enumerate(eligible.iterrows(), start=1):
            confluence = row['confluence']
            if confluence == 'EXTENDED':
                color = colors.lightgreen
            elif confluence == 'BALANCED':
                color = colors.lightyellow
            elif confluence == 'WEAK':
                color = colors.lightcoral
            else:
                color = colors.white
            
            # Apply color to confluence column (column 4, shifted due to new Current % column)
            style_commands.append(('BACKGROUND', (4, i), (4, i), color))
    
    table.setStyle(TableStyle(style_commands))
    
    elements.append(table)
    elements.append(Spacer(1, 0.3*inch))
    
    # Add execution plan
    exec_text = """
    <b>Execution Plan</b><br/>
    <br/>
    • Place limit orders only at specified Zone prices (no chasing current prices)<br/>
    • Wait for pullbacks to target zones before entering positions<br/>
    • Other stocks: No action (insufficient signal strength or poor technical setup)
    """
    elements.append(Paragraph(exec_text, styles['Normal']))
    elements.append(Spacer(1, 0.2*inch))
    
    # Add footer
    footer_text = f"<font size=8>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</font>"
    elements.append(Paragraph(footer_text, styles['Normal']))
    
    # Build PDF
    doc.build(elements)
    print(f"✅ Buy Summary PDF saved to Downloads: {pdf_path}")

# Generate PDF with starting cash parameter
export_buy_summary_to_pdf(df, starting_cash=104967)

✅ Buy Summary PDF saved to Downloads: C:\Users\karms\Downloads\buy_summary_20260105_0003.pdf


In [64]:
# Cell 5: Export Results
from datetime import datetime

# Add timestamp to avoid overwriting
timestamp = datetime.now().strftime("%Y%m%d_%H%M")
output_file = Path.home() / 'Downloads' / f'batch_results_{timestamp}.csv'

# Exclude baskets from CSV export
df_export = df[~df['ticker'].str.startswith('[')]

# Save directly to Downloads folder
df_export.to_csv(output_file, index=False)
print(f"✅ Results saved to Downloads: {output_file}")

✅ Results saved to Downloads: C:\Users\karms\Downloads\batch_results_20260105_0005.csv


In [25]:
# Cell 5: Cleanup Old Results
from pathlib import Path

def cleanup_old_results(keep_latest=1):
    """Keep only the most recent batch result file"""
    batch_files = sorted(
        ROOT.glob('batch_results_*.csv'),
        key=lambda p: p.stat().st_mtime,
        reverse=True
    )
    
    for old_file in batch_files[keep_latest:]:
        print(f"🗑️  Deleting: {old_file.name}")
        old_file.unlink()
    
    print(f"✅ Kept {min(len(batch_files), keep_latest)} most recent file(s)")

# Auto-cleanup after each run:
cleanup_old_results(keep_latest=1)

✅ Kept 1 most recent file(s)
